이 파일의 소스코드는 아래의 레포지토리를 참고했다.

### Reference:
- https://github.com/rakeshchada/gpt2-singular-plural

In [2]:
import torch
import numpy as np
from tqdm import trange
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from generation import generate

In [3]:
MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop
length = 20

In [4]:
temperature = 1.0
top_k = 0.0
top_p = 0.9

In [5]:
MODEL_CLASSES = {
    'gpt2': (GPT2LMHeadModel, GPT2Tokenizer),
}

In [6]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.device_count() > 0:
        torch.cuda.manual_seed_all(seed)

In [7]:
device = torch.device('cpu')
seed = 42

In [8]:
model_type = 'gpt2'
model_name_or_path = 'gpt2'
model_class, tokenizer_class = MODEL_CLASSES[model_type]

In [9]:
model_class, tokenizer_class

(transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel,
 transformers.models.gpt2.tokenization_gpt2.GPT2Tokenizer)

In [10]:
tokenizer = tokenizer_class.from_pretrained(model_name_or_path)
model = model_class.from_pretrained(model_name_or_path)
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [11]:
set_seed(seed)

### singular test

In [12]:
verbs = []
with open('five_verbs.txt', 'r') as f:
    for line in f:
        verbs.append(line.lower().replace('\n',''))

In [13]:
sum([m.sum() for m in model.parameters()])

tensor(-62119.8867, grad_fn=<AddBackward0>)

In [14]:
plurals = []
for verb in verbs:
    prompt = f"play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . {verb} ="
    plural = generate(model, tokenizer, prompt)
    plural = plural.split(".")[0]
    plurals.append(plural)

In [15]:
prompt

'play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . jump ='

In [16]:
for v, p in zip(verbs, plurals):
    print(f'{v} -> {p}')

click ->  clicked
work ->  worked
walk ->  walked
run ->  run
jump ->  jump


In [18]:
for verb in verbs:
    prompt = f"play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . {verb} ="
    print(prompt)

play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . click =
play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . work =
play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . walk =
play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . run =
play = played . sing = sang . view = viewed . act = acted . say = said . type = typed . note = noted . see = saw . clean = cleaned . tell = told . jump =
